In [ ]:
# ✅ Notebook C – c-curadoria.ipynb
# Função: Início da curadoria de arquivos únicos, a partir do bucket 'storage-unique'
# Etapa inicial: seleção de projeto e listagem de imagens para pré-processamento

# 📦 Importações e Conexões Iniciais
from minio import Minio
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🔗 Configuração do MinIO
MINIO_ENDPOINT = "minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "senhasegura"
BUCKET_ORIGIN = "storage-unique"
BUCKET_CURATED = "storage-curated"

# 🔗 Configuração do PostgreSQL
PG_HOST = "postgres_db"
PG_PORT = "5432"
PG_DATABASE = "postgres"
PG_USER = "postgres"
PG_PASSWORD = "senhasegura"

# 🔌 Conectando ao MinIO
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)
if not client.bucket_exists(BUCKET_CURATED):
    client.make_bucket(BUCKET_CURATED)

# 🔌 Conexão via SQLAlchemy
PG_URL = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"
engine = create_engine(PG_URL)

# 🔍 Buscar projetos ativos para dropdown
def fetch_projects():
    query = """
        SELECT prefix, project_name
        FROM projects_registry
        WHERE active = TRUE
        ORDER BY project_name;
    """
    df = pd.read_sql(query, engine)
    if df.empty:
        return []
    return [f"{row.prefix} - {row.project_name}" for _, row in df.iterrows()]

# 🎛️ Interface inicial
project_dropdown = widgets.Dropdown(
    options=fetch_projects(),
    description="Projeto:",
    layout=widgets.Layout(width='60%')
)
run_button = widgets.Button(
    description="Listar Imagens",
    button_style='success'
)
output = widgets.Output()

# ▶️ Callback da listagem
@output.capture()
def on_run_clicked(b):
    clear_output()
    if not project_dropdown.value:
        print("❌ Selecione um projeto.")
        return

    prefix = project_dropdown.value.split(" - ")[0]
    print(f"🔍 Listando imagens do projeto '{prefix}' no bucket '{BUCKET_ORIGIN}'...")

    query = text("""
        SELECT full_path, filename FROM storage_audit
        WHERE prefix = :prefix AND (filename ILIKE '%%.png' OR filename ILIKE '%%.jpg')
    """)
    with engine.connect() as con:
        rows = con.execute(query, {"prefix": prefix}).fetchall()
        if not rows:
            print("⚠️ Nenhuma imagem encontrada.")
        else:
            print(f"🖼️ {len(rows)} imagens encontradas:")
            for path, name in rows[:10]:
                print(f" • {path}")

# 🧼 Limpa bindings anteriores e conecta botão
run_button._click_handlers.callbacks = []
run_button.on_click(on_run_clicked)
display(project_dropdown, run_button, output)


Dropdown(description='Projeto:', layout=Layout(width='60%'), options=('FDL - FIAP - DEEP LEARNING',), value='F…

Button(button_style='success', description='Listar Imagens', style=ButtonStyle())

Output()

In [1]:
# ✅ Notebook C – c-curadoria.ipynb
# Função: Selecionar projeto e exibir cardápio de arquivos únicos disponíveis por tipo
# Etapa: Classificação e roteamento para módulos especializados (.py) de curadoria

from minio import Minio
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🔗 Configuração do MinIO
MINIO_ENDPOINT = "minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "senhasegura"
BUCKET_ORIGIN = "storage-unique"
BUCKET_CURATED = "storage-curated"

# 🔗 Configuração do PostgreSQL
PG_HOST = "postgres_db"
PG_PORT = "5432"
PG_DATABASE = "postgres"
PG_USER = "postgres"
PG_PASSWORD = "senhasegura"

# 🔌 Conexões
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)
if not client.bucket_exists(BUCKET_CURATED):
    client.make_bucket(BUCKET_CURATED)

PG_URL = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"
engine = create_engine(PG_URL)

# 🎯 Função para determinar tipo de arquivo com base na extensão
EXT_TIPO = {
    "imagem": [".png", ".jpg", ".jpeg"],
    "numerico": [".csv", ".xls", ".xlsx"],
    "texto": [".pdf", ".txt", ".docx"],
}

def classificar_tipo(ext):
    ext = ext.lower()
    for tipo, lista in EXT_TIPO.items():
        if ext in lista:
            return tipo
    return "vetor"  # categoria genérica para outros formatos

# 🎛️ Widgets principais
project_dropdown = widgets.Dropdown(
    options=[],
    description="Projeto:",
    layout=widgets.Layout(width='60%')
)
tipo_dropdown = widgets.Dropdown(
    options=[],
    description="Tipo:",
    layout=widgets.Layout(width='40%')
)
run_button = widgets.Button(
    description="Carregar Cardápio",
    button_style='success'
)
output = widgets.Output()

# 🔍 Buscar projetos ativos
def fetch_projects():
    df_proj = pd.read_sql("""
        SELECT prefix, project_name
        FROM projects_registry
        WHERE active = TRUE
        ORDER BY project_name;
    """, engine)
    return [f"{row.prefix} - {row.project_name}" for _, row in df_proj.iterrows()]

# ▶️ Callback principal
@output.capture()
def on_run_clicked(b):
    clear_output()
    if not project_dropdown.value:
        print("❌ Selecione um projeto.")
        return

    prefix = project_dropdown.value.split(" - ")[0]
    print(f"📊 Analisando arquivos do projeto '{prefix}'...")

    query = text("""
        SELECT full_path, filename FROM storage_audit
        WHERE prefix = :prefix
    """)
    with engine.connect() as con:
        rows = con.execute(query, {"prefix": prefix}).fetchall()

    if not rows:
        print("⚠️ Nenhum arquivo encontrado para este projeto.")
        return

    df = pd.DataFrame(rows, columns=["full_path", "filename"])
    df["ext"] = df["filename"].apply(lambda x: x.split(".")[-1].lower())
    df["tipo"] = df["ext"].apply(lambda x: classificar_tipo("." + x))

    tipos = df["tipo"].value_counts().to_dict()
    tipo_dropdown.options = [f"{k} ({v})" for k, v in tipos.items()]
    display(tipo_dropdown)

    print("\n📋 Amostras por tipo:")
    for tipo in tipos.keys():
        total = tipos[tipo]
        print(f"\n🔹 {tipo.upper()} ({total} arquivos)")
        sample = df[df["tipo"] == tipo].head(10)
        for path in sample["full_path"]:
            print(f" • {path}")

# 🧼 Limpar e conectar eventos
run_button._click_handlers.callbacks = []
run_button.on_click(on_run_clicked)
project_dropdown.options = fetch_projects()

# ▶️ Exibir interface
ui = widgets.VBox([project_dropdown, run_button, output])
display(ui)
